##  Exploratory Data Analysis For Positive Growth Cultures

In [4]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
import os 

# ### THIS IS MEANT TO RUN ON NERO - NEEDS TO BE CHANGED IF YOU RUN LOCALLY
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/ccorbin/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'mining-clinical-decisions' 
%load_ext google.cloud.bigquery

# from google.cloud import bigquery
# client=bigquery.Client()

In [8]:
%%bigquery
# Stratify Positive Culture Encounters By The Number of Organisms that Grew in the encounter
SELECT num_bugs, COUNT (DISTINCT pat_enc_csn_id_coded) num_csns
FROM 
  (SELECT cr.pat_enc_csn_id_coded, COUNT (DISTINCT cr.sens_organism_sid) num_bugs
   FROM `mining-clinical-decisions.conor_db.abx_ast_long` cr
   GROUP BY cr.pat_enc_csn_id_coded) t
GROUP BY num_bugs
ORDER BY num_bugs

,num_bugs,num_csns
0,1,6754
1,2,1683
2,3,204
3,4,53
4,5,9
5,6,3
6,7,2


In [10]:
%%bigquery
# Get types of organisms that grow stratified by ER that grew them
# Top 15 per ER
WITH dep_id as (
SELECT 
adt.pat_enc_csn_id_coded,
FIRST_VALUE(adt.department_id) OVER (PARTITION BY adt.pat_enc_csn_id_coded ORDER BY adt.effective_time_jittered_utc) department_id
FROM `mining-clinical-decisions.conor_db.abx_ast_long` 
INNER JOIN `shc_core.adt` adt
USING (pat_enc_csn_id_coded)
),

all_bugs as (
SELECT CASE WHEN department_name in ('ADULT EMERGENCY DEPARTMENT', 'PEDIATRIC EMERGENCY DEPARTMENT') THEN "SHC ED" ELSE department_name END department_name,
organism, COUNT (DISTINCT pat_enc_csn_id_coded) num_csns
FROM 
  (SELECT DISTINCT cr.pat_enc_csn_id_coded, cr.organism, dm.department_name
  FROM `mining-clinical-decisions.conor_db.abx_ast_long` cr
  INNER JOIN dep_id
  USING (pat_enc_csn_id_coded)
  INNER JOIN `shc_core.dep_map` dm
  USING (department_id)) t
GROUP BY department_name, organism
-- ORDER BY department_name, num_csns DESC
),

topten as (
SELECT *, ROW_NUMBER() 
over (
    PARTITION BY department_name 
    order by num_csns DESC
) AS RowNo 
FROM all_bugs
)

SELECT * FROM topten WHERE RowNo <= 15
ORDER BY department_name, RowNo



,department_name,organism,num_csns,RowNo
0,SHC ED,ESCHERICHIA COLI,3726,1
1,SHC ED,KLEBSIELLA PNEUMONIAE,814,2
2,SHC ED,ENTEROCOCCUS SPECIES,761,3
3,SHC ED,STAPHYLOCOCCUS AUREUS,744,4
4,SHC ED,PROTEUS MIRABILIS,376,5
5,SHC ED,PSEUDOMONAS AERUGINOSA,362,6
6,SHC ED,STAPH AUREUS {MRSA},161,7
7,SHC ED,ENTEROCOCCUS FAECALIS,158,8
8,SHC ED,STREPTOCOCCUS AGALACTIAE (GROUP B),144,9
9,SHC ED,KLEBSIELLA OXYTOCA,132,10
